<a href="https://colab.research.google.com/github/kittyschulz/EDA-Waymo-Open-Dataset/blob/master/Waymo_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf waymo-od > /dev/null
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git branch -a
!cd waymo-od && git checkout remotes/origin/r1.0
!pip3 install --upgrade pip

Cloning into 'waymo-od'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 367 (delta 0), reused 3 (delta 0), pack-reused 363
Receiving objects: 100% (367/367), 14.00 MiB | 18.26 MiB/s, done.
Resolving deltas: 100% (192/192), done.
* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/master
  remotes/origin/r1.0
  remotes/origin/r1.0-tf1.15
Note: checking out 'remotes/origin/r1.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at f4ae13f Merge branch 'master' of sso://user/chaiy/waymo-open-dataset into r1.0
Requiremen

In [36]:
!pip3 install waymo-open-dataset
import os
import tensorflow as tf
import math
import numpy as np
import pandas as pd
import itertools
import collections

tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset import label_pb2

In [8]:
FILENAME = '/content/waymo-od/tutorial/frames'
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    break
    
type(frame)

waymo_open_dataset.dataset_pb2.Frame

In [0]:
def object_type_name(x):
  return label_pb2.Label.Type.Name(x)

In [38]:
object_count = collections.Counter([object_type_name(x.type) for x in frame.laser_labels])
print(object_count)

Counter({'TYPE_PEDESTRIAN': 21, 'TYPE_VEHICLE': 13, 'TYPE_SIGN': 12})


In [29]:
# Expolre the frame proto
for x in frame.context.stats.laser_object_counts:
  print(type(object_type_name(x.type)))
  print(type(x.count))

<class 'str'>
<class 'int'>
<class 'str'>
<class 'int'>
<class 'str'>
<class 'int'>


In [0]:
# stats = ['laser_object_counts_Vehicle', 'laser_object_counts_Pedestrian', 'laser_object_counts_Sign', 'camera_object_counts_Vehicle', 'camera_object_counts_Pedestrian', 'time_of_day', 'location', 'weather']
stats = ['time_of_day', 'location', 'weather']


In [39]:
def extract_frame_data(frame):
  frame_data = {                     
       'time_of_day': frame.context.stats.time_of_day,
       'location': frame.context.stats.location,
       'weather': frame.context.stats.weather
    }
  object_counts = {object_type_name(x.type): x.count for x in frame.context.stats.laser_object_counts}
  frame_data.update(object_counts)
  
  object_count = collections.Counter([object_type_name(x.type) for x in frame.laser_labels])
  print(object_count)
  
  return frame_data


frame_df = pd.DataFrame()
FILENAME = '/content/waymo-od/tutorial/frames'
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
for data in dataset:
    frame = open_dataset.Frame()
    print('Parsing frame')
    frame.ParseFromString(bytearray(data.numpy()))
    frame_data = extract_frame_data(frame)
    frame_df = frame_df.append(frame_data, ignore_index=True)

frame_df.head()

Parsing frame
Counter({'TYPE_PEDESTRIAN': 21, 'TYPE_VEHICLE': 13, 'TYPE_SIGN': 10})
Parsing frame
Counter({'TYPE_PEDESTRIAN': 21, 'TYPE_VEHICLE': 13, 'TYPE_SIGN': 12})


,TYPE_PEDESTRIAN,TYPE_SIGN,TYPE_VEHICLE,location,time_of_day,weather
0,21.0,10.0,13.0,location_sf,Day,sunny
1,21.0,12.0,13.0,location_sf,Day,sunny
